# Coarse-graining with PCCA+

In [ ]:
import numpy as np
import sktime

In [ ]:
data = sktime.data.ellipsoids(laziness=0.97, seed=32).observations(n_steps=5000)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(*data.T);

In [ ]:
clustering = sktime.clustering.KmeansClustering(n_clusters=50, fixed_seed=13) \
    .fit(data).fetch_model()
assignments = clustering.transform(data)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.scatter(*data.T, c=assignments/np.max(assignments))
ax.scatter(*clustering.cluster_centers.T, marker="*", c='red')

In [ ]:
msm = sktime.markov.msm.MaximumLikelihoodMSM() \
    .fit(assignments, lagtime=1).fetch_model()

In [ ]:
msm.transition_matrix.shape

In [ ]:
pcca = msm.pcca(n_metastable_sets=2)

In [ ]:
pcca.coarse_grained_transition_matrix

In [ ]:
crisp_memberships = pcca.memberships.argmax(axis=1)

f, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.scatter(*data.T, c=assignments)
ax.scatter(*clustering.cluster_centers.T, marker="*", 
            c=['red' if m == 0 else 'cyan' for m in crisp_memberships])